<a href="https://colab.research.google.com/github/aiswarya-1422/AI-SMART-SEARCH-for-Kerala-GOVT-/blob/main/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import json
import os

# Path to your uploaded file
zip_path = "/content/processed_json_en.zip"
extract_path = "/content/processed_json_en"

# Unzip the uploaded JSON files
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Collect all JSON files inside the extracted folder
json_files = [os.path.join(extract_path, f) for f in os.listdir(extract_path) if f.endswith(".json")]

# Load the summarized content from each JSON file
documents = []
for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # If your JSON has a key like "summary" or "content", update this line accordingly
        text_list = data.get("summary") or data.get("content") or [] # Get the list, default to empty list
        text = " ".join(text_list) # Join the list elements into a single string

        if text.strip():
            documents.append(text)

print(f"✅ Loaded {len(documents)} summarized documents.")
if documents: # Check if documents list is not empty before trying to access an element
    print(documents[0][:500])  # Preview first 500 characters of the first document
else:
    print("No documents were loaded.")

✅ Loaded 451 summarized documents.
 The Director of Health Services has reported that a large number of costly equipment are Kept away for want of repairs and proper maintenance . The repairs are-done bv officers in charge of medical institutions IN a “case - to ~ case” basis after obtaining administrative and technical sanction .  The Director of Health Services has enhanced powers to issue administrative sanction for repairs and maintenance of equipment . Chain Officer has the power to issue technical sanction for repair and ma


In [ ]:
!pip install sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all documents
embeddings = model.encode(documents, show_progress_bar=True)

# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

print("✅ Semantic search index created successfully!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

✅ Semantic search index created successfully!


In [ ]:
def semantic_search(query, top_k=3):
    query_vector = model.encode([query])
    D, I = index.search(query_vector, top_k)
    results = []
    for j, i in enumerate(I[0]):
        results.append({
            "rank": j+1,
            "similarity_score": float(D[0][j]),
            "text": documents[i][:400] + "..."
        })
    return results

# Example usage
query = "budget allocation for education sector"
results = semantic_search(query, top_k=3)

for r in results:
    print(f"\n🔹 Rank {r['rank']} | Score: {r['similarity_score']:.4f}\n{r['text']}")



🔹 Rank 1 | Score: 0.9651
 The general guidelines on MLq. SDF scheme, as modified from time to rime have permitted construction of buildings, purchase of audio-visual equipments of educational nature,/providing computers and purchase of library books . The PAC in its 25'h report (2011-14) suggested Government to stop the practice of allocation of funds to the Aided/Un-aided Educational Institutions under the scheme . After...

🔹 Rank 2 | Score: 1.0520
 The Public Accounts Committee in the 1401 Report (2008-2011) has recommended to introduce Performance Budgeting in all the Departments dealing with developmental activities on a time bound basis . The exercise will focus on developmental schemes and related expenditure only and not to routine administrative expenses .  The main objective is to establish a one to . one correspondence between finan...

🔹 Rank 3 | Score: 1.1130
 Kerala Finance (Budget Wing - F) Department CIRCular No.15/2019/Fin Dated: 16.02,2019, Thiruvaranthapuram . Sub- 

In [ ]:
faiss.write_index(index, "/content/semantic_index.faiss")
np.save("/content/doc_embeddings.npy", embeddings)
print("✅ Index and embeddings saved.")


✅ Index and embeddings saved.
